In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


#### Read Data In

In [2]:
import db.knowhere_db as kdb

In [3]:
reader = kdb.Reader('knowhere')
g1 = reader.get_dataframe_pivoted(collection='iphone_test3', username='glen', min_date='2017-3-20', 
                                  max_date='2017-03-21')

In [4]:
g1.head(20)

,Microphone Left Channel Level,GPS Horizontal Accuracy,Microphone Right Channel Level,GPS Longitude,GPS Vertical Accuracy,Altimeter (Barometer) Pressure,GPS Latitude,GPS Altitude,Altimeter (Barometer) Relative Altitude,Gravity z,...,Gravity x,Gyrometer y,Gyrometer z,Acceleration z,Acceleration x,Acceleration y,Gyrometer x,Magnetometer z,Magnetometer y,Magnetometer x
2017-03-20 07:33:32,NaN,65.0,NaN,-73.753674,10.0,NaN,40.736696,49.925198,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 07:33:33,NaN,65.0,NaN,-73.753620,10.0,101.280930,40.736642,49.617031,0.000000,0.569031,...,0.069859,-0.212767,0.489268,-1.372254,-0.152821,0.563072,-1.204461,-448.443573,138.553955,-53.052246
2017-03-20 07:33:34,NaN,NaN,NaN,NaN,NaN,101.281799,NaN,NaN,-0.072419,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 07:33:35,NaN,NaN,NaN,NaN,NaN,101.282677,NaN,NaN,-0.145844,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 07:33:36,NaN,NaN,NaN,NaN,NaN,101.284218,NaN,NaN,-0.274084,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 07:33:37,NaN,50.0,NaN,-73.753751,8.0,101.285233,40.736369,45.059113,-0.358571,-0.015207,...,-0.240759,NaN,NaN,-0.714468,0.377050,0.415137,NaN,NaN,NaN,NaN
2017-03-20 07:33:38,-2.53365,10.0,-2.53365,-73.753784,3.0,101.286613,40.736889,48.031158,-0.473734,NaN,...,NaN,-0.004173,-0.025536,NaN,NaN,NaN,-0.010636,NaN,NaN,NaN
2017-03-20 07:33:39,NaN,10.0,NaN,-73.753771,3.0,NaN,40.736817,48.538971,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 07:33:40,NaN,10.0,NaN,-73.753745,3.0,101.287239,40.736894,48.434418,-0.525531,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-439.615295,118.981277,-50.648590
2017-03-20 07:33:41,NaN,10.0,NaN,-73.753743,3.0,101.287621,40.736893,48.664825,-0.556710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#glen1 = pd.read_csv("Knowhere/data/iPhone/incoming/Glen_03202017_Morning.csv", error_bad_lines=False, index_col=0)
#glen1.index = pd.to_datetime(glen1.index)


In [6]:
f = open("../data/iPhone/incoming/Glen_03202017_Morning.txt","r")
glen_3_19 = f.read()
f.close()

In [7]:
glen_3_19

'[(7:36:20, I_C),(7:41:55,   W),(7:49:01, I_C),(7:55:53,   W),\\\n (7:56:53,   S),(8:00:23, T_D),(8:16:15, T_T),(8:21:05, T_D),\\\n (8:30:30, T_E),(8:32:50,   W),(8:36:40, E_U),(8:38:10,   W)\n\n\n'

#### Necessary Input

In [8]:
glen_3_20_events = [("7:36:20", "D"),("7:41:55", "W"),("7:49:01", "D"),("7:52:45","W"),("7:55:45", "S_D"),
             ("7:55:53", "P_W"),("7:56:53", "P_S"),("8:00:23", "T_B"), ("8:16:15", "T_T"),("8:21:05", "T_B"),
             ("8:30:30", "T_E"),("8:31:15", "S_U"),("8:31:20","W"),("8:32:05", "S_U"),("8:32:10","W"),
             ("8:32:35", "S_U"),("8:32:50", "W"),("8:36:40", "E_U"),("8:37:50", "W"),("8:30:10", "W")]

#### Functions for Classifying

In [9]:
def add_acceleration_magnitude(df):
    "Adds Acceleration magnitude to a dataframe. First 3 columns must be acceleration vectors. Must be 3D."
    from math import sqrt

    accel_mag = []
    
    for row in range(len(df)):
        accel_mag.append(sqrt(df.iloc[row, 0]**2 + df.iloc[row, 1]**2 + df.iloc[row, 2]**2))

    df['accel_mag'] = accel_mag
    return 'finished!'

In [10]:
def rel_time_to_real_time(df, day, start_time):
    from pandas.tseries.offsets import DateOffset
    df.index = [pd.Timestamp(day + " " + start_time) + DateOffset(seconds=x) for x in df.index]
    return "Finished!"

In [11]:
def add_classifications(df, day, events):
    '''Pass events as a list of tuples (datetime, eventcode) to classify different events '''
    from pandas.tseries.offsets import DateOffset
    
    for event in events:
        df.loc[day + " " + event[0], 'classification'] = event[1]
        if event[1] == "T_S":
            df.loc[str((pd.Timestamp(day + " " + event[0])+ pd.DateOffset(seconds=40))), 'classification'] = "T_B"
    df = df.fillna(method='ffill', inplace = True)
    
    return "Finished!"

In [12]:
def label_raw_iphone(df, day, events, username = "glen", accel_mag = False):
    df.index = pd.to_datetime(df.index)
    if accel_mag == True:
        add_acceleration_magnitude(df)
    df['classification'] = None
    add_classifications(df, day, events)
    df.to_csv("Labeled_" + username + day)
    return df

In [13]:
#don't run until path is figured out or df.to_csv is manually changed in label_raw_iphone function
label_raw_iphone(g1, "2017-03-20", glen_3_20_events)

,Microphone Left Channel Level,GPS Horizontal Accuracy,Microphone Right Channel Level,GPS Longitude,GPS Vertical Accuracy,Altimeter (Barometer) Pressure,GPS Latitude,GPS Altitude,Altimeter (Barometer) Relative Altitude,Gravity z,...,Gyrometer y,Gyrometer z,Acceleration z,Acceleration x,Acceleration y,Gyrometer x,Magnetometer z,Magnetometer y,Magnetometer x,classification
2017-03-20 07:33:32,NaN,65.0,NaN,-73.753674,10.0,NaN,40.736696,49.925198,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2017-03-20 07:33:33,NaN,65.0,NaN,-73.753620,10.0,101.280930,40.736642,49.617031,0.000000,0.569031,...,-0.212767,0.489268,-1.372254,-0.152821,0.563072,-1.204461,-448.443573,138.553955,-53.052246,None
2017-03-20 07:33:34,NaN,65.0,NaN,-73.753620,10.0,101.281799,40.736642,49.617031,-0.072419,0.569031,...,-0.212767,0.489268,-1.372254,-0.152821,0.563072,-1.204461,-448.443573,138.553955,-53.052246,None
2017-03-20 07:33:35,NaN,65.0,NaN,-73.753620,10.0,101.282677,40.736642,49.617031,-0.145844,0.569031,...,-0.212767,0.489268,-1.372254,-0.152821,0.563072,-1.204461,-448.443573,138.553955,-53.052246,None
2017-03-20 07:33:36,NaN,65.0,NaN,-73.753620,10.0,101.284218,40.736642,49.617031,-0.274084,0.569031,...,-0.212767,0.489268,-1.372254,-0.152821,0.563072,-1.204461,-448.443573,138.553955,-53.052246,None
2017-03-20 07:33:37,NaN,50.0,NaN,-73.753751,8.0,101.285233,40.736369,45.059113,-0.358571,-0.015207,...,-0.212767,0.489268,-0.714468,0.377050,0.415137,-1.204461,-448.443573,138.553955,-53.052246,None
2017-03-20 07:33:38,-2.53365,10.0,-2.53365,-73.753784,3.0,101.286613,40.736889,48.031158,-0.473734,-0.015207,...,-0.004173,-0.025536,-0.714468,0.377050,0.415137,-0.010636,-448.443573,138.553955,-53.052246,None
2017-03-20 07:33:39,-2.53365,10.0,-2.53365,-73.753771,3.0,101.286613,40.736817,48.538971,-0.473734,-0.015207,...,-0.004173,-0.025536,-0.714468,0.377050,0.415137,-0.010636,-448.443573,138.553955,-53.052246,None
2017-03-20 07:33:40,-2.53365,10.0,-2.53365,-73.753745,3.0,101.287239,40.736894,48.434418,-0.525531,-0.015207,...,-0.004173,-0.025536,-0.714468,0.377050,0.415137,-0.010636,-439.615295,118.981277,-50.648590,None
2017-03-20 07:33:41,-2.53365,10.0,-2.53365,-73.753743,3.0,101.287621,40.736893,48.664825,-0.556710,-0.015207,...,-0.004173,-0.025536,-0.714468,0.377050,0.415137,-0.010636,-439.615295,118.981277,-50.648590,None


#### Classifying Data